In [ ]:
# get n occurrences based on number of series
# this overlaps with Arun's file that computes counts, can be used to compare
# note that gender and supplemental concept records (SCR) have no treenumber and will be excluded
# this takes 1-2 hours to run

import pandas as pd
import numpy as np
import os

# load tree
path = '../../data/final'
tree_file = 'mesh.pkl'
trdf = pd.read_pickle(os.path.join(path, tree_file))

# load tags
tag_file ='geo_filtered.pkl'
tadf = pd.read_pickle(os.path.join(path, tag_file))
tadf['year'] = (tadf.date.str[:4]).astype(int) # year
tadf = pd.merge(tadf,trdf[['mesh_id', 'mesh_treenumbers']], on='mesh_id', how='left')

tadf['for_counts'] = 1

def get_unique_occurrences(tadf,mesh_and_tree, imesh):
    tree_list = mesh_and_tree[np.isin(mesh_and_tree.mesh_id,imesh)].mesh_treenumbers.to_list()
    geo_list = []
    for itree in tree_list: 
        geo_list_aux = tadf.loc[tadf.mesh_treenumbers.str[:len(itree)]==itree,'geo_id'].to_list()
        geo_list+=geo_list_aux
    n_geo = len(np.unique(np.array(geo_list)))
    return n_geo

mesh_and_tree = tadf.groupby(['mesh_id', 'mesh_treenumbers']).for_counts.mean().reset_index()
unique_mesh = tadf.mesh_id.unique()

n_geo_list = []
for imesh in unique_mesh:
    n_geo = get_unique_occurrences(tadf,mesh_and_tree, imesh)
    n_geo_list.append(n_geo)

n_mentions_by_disease_descendents = pd.DataFrame.from_dict({'mesh_id':pd.Series(unique_mesh) , 'n': pd.Series(n_geo_list)})
n_mentions_by_disease_descendents = n_mentions_by_disease_descendents.sort_values('n')
n_mentions_by_disease_descendents = n_mentions_by_disease_descendents.loc[n_mentions_by_disease_descendents.n!=0]
n_mentions_by_disease_descendents = n_mentions_by_disease_descendents.sort_values('n', ascending=False).reset_index(drop=True)
n_mentions_by_disease_descendents.to_pickle('../../data/final/meshids_rankedby_NSeries.pkl')